**자립 정보 ON 지원 사업 조회에 있는 지원 사업 마다 txt와 이미지를 저장하는 코드**


selenium을 이용해서 크롤링해오고 현재는 10개의 지원 사업만 저장하게 했습니다.

**Google Drive API를 이용해서 img와 txt를 drive에 저장해보자**
* API 사용 설정과 OAuth 동의, 데스크톱 어플리케이션의 사용자 인증 정보 승인
* credentials.json 저장
* python용 Google 클라이언트 라이브러리 설치

In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import requests
import random
import time
import io
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google.oauth2.service_account import Credentials

# 서비스 계정 키 파일 경로
SERVICE_ACCOUNT_FILE = 'service_account.json'

# Google Drive API 인증 (서비스 계정)
def authenticate_google_drive():
    credentials = Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=['https://www.googleapis.com/auth/drive']
    )
    return build('drive', 'v3', credentials=credentials)

# Google Drive 공유 폴더 ID
FOLDER_ID = '1GwbcEfpaCbcbkhBpXjR4GOO6UFVjceiz'

# 파일 업로드 함수
def upload_to_drive(service, file_name, file_data, mime_type):
    file_metadata = {
        'name': file_name,
        'parents': [FOLDER_ID]
    }
    media = MediaIoBaseUpload(io.BytesIO(file_data), mimetype=mime_type)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"Uploaded {file_name} to shared folder with ID: {file.get('id')}")

# WebDriver 설정
driver = webdriver.Chrome()

# 메인 페이지로 이동
driver.get("https://jaripon.ncrc.or.kr/home/kor/main.do")

# 경고창 처리
try:
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert.accept()
except Exception:
    pass

# JavaScript로 페이지 이동
script = "fn_menu_move('/home/kor/support/projectMng/index.do', '3');"
driver.execute_script(script)

# Google Drive API 인증
drive_service = authenticate_google_drive()

try:
    gallery_list = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
    )

    for idx in range(10):
        try:
            li_elements = gallery_list.find_elements(By.CSS_SELECTOR, "li > a")
            a_tag = li_elements[idx]
            driver.execute_script("arguments[0].click();", a_tag)

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            editor_view = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.editor_view"))
            )

            # 텍스트 업로드
            text_content = editor_view.text
            if text_content:
                text_data = text_content.encode('utf-8')
                text_file_name = f"content_{idx + 1}.txt"
                upload_to_drive(drive_service, text_file_name, text_data, "text/plain")

            # 이미지 업로드
            image_elements = editor_view.find_elements(By.TAG_NAME, "img")
            for img_idx, img in enumerate(image_elements):
                img_url = img.get_attribute("src")
                if img_url:
                    img_data = requests.get(img_url).content
                    image_file_name = f"content_{idx + 1}_image_{img_idx + 1}.jpg"
                    upload_to_drive(drive_service, image_file_name, img_data, "image/jpeg")

            driver.back()
            driver.execute_script(script)
            gallery_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
            )
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing link {idx + 1}: {e}")
            break

except Exception as e:
    print(f"Error finding gallery_list or list items: {e}")

driver.quit()


Uploaded content_1.txt to shared folder with ID: 1j9gx3Nf0OTqXO6PwYRX6Ln7mwj3_LOLC
Uploaded content_1_image_1.jpg to shared folder with ID: 1F3L-Gje_jKxzAziqv8lyIWeJ9ZIX5sd0
Uploaded content_1_image_2.jpg to shared folder with ID: 1-U2dASLUWHDd6sQFbmjheoz12RIox9bf
Uploaded content_1_image_3.jpg to shared folder with ID: 16Nm-Ll2AQ7T1-06d1vYl1N2wRkQRdtM9
Uploaded content_1_image_4.jpg to shared folder with ID: 1ulnxjaWDERtLle8riJjuYB_ifHzdDJ2c
Uploaded content_1_image_5.jpg to shared folder with ID: 1Gw-FVBZoZSWggjPsF21NRpRfKVHelGNX
Uploaded content_1_image_6.jpg to shared folder with ID: 1zH33ebbQcrtvTqUzA3-5Y81o1oPJO9gE
Uploaded content_1_image_7.jpg to shared folder with ID: 1anPR1AJkY1IyQld_ap-JWcgFLCGF51Y7
Uploaded content_1_image_8.jpg to shared folder with ID: 1iSNDZmtyb5A_oaB72pdMFVyA46z8KGUf
Uploaded content_1_image_9.jpg to shared folder with ID: 1JA84fHrofvRUMtvGDL8XBjO4pjZQFpgJ
Uploaded content_1_image_10.jpg to shared folder with ID: 1AlsYXW_kOBNZMffQV16vdf67GfuSFhU5
Upload

In [1]:
import os
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from selenium import webdriver

# WebDriver 설정 (Chrome 예시)
driver = webdriver.Chrome()

# 메인 페이지로 이동
driver.get("https://jaripon.ncrc.or.kr/home/kor/main.do")

# 경고창 처리 (비정상적 접근 알림을 닫음)
try:
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert.accept()
    print("Alert accepted.")
except Exception as e:
    print(f"No alert found or error occurred: {e}")

# 자립정보 조회 페이지로 이동 (JavaScript 사용)
script = "fn_menu_move('/home/kor/support/projectMng/index.do', '3');"
driver.execute_script(script)

# 페이지 로딩 및 `div.gallery_list > ul.list` 요소 확인
try:
    gallery_list = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
    )
    print("gallery_list found.")

    # 각 `li > a` 요소 탐색 및 텍스트/이미지 저장 반복
    for idx in range(10):  # 예: 최대 10개의 항목 처리
        try:
            # 현재 `li > a` 요소들 로드
            li_elements = gallery_list.find_elements(By.CSS_SELECTOR, "li > a")

            # a 태그 클릭하여 새 페이지로 이동
            a_tag = li_elements[idx]
            driver.execute_script("arguments[0].click();", a_tag)

            # 새 페이지가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            # <div class="editor_view">에서 텍스트 및 이미지 URL 가져오기
            editor_view = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.editor_view"))
            )

            # 텍스트 내용 가져와서 저장
            text_content = editor_view.text
            text_filename = f"content_{idx + 1}.txt"
            with open(text_filename, "w", encoding="utf-8") as file:
                file.write(text_content)
            print(f"Saved text content to {text_filename}.")

            # 이미지 다운로드 및 저장
            image_elements = editor_view.find_elements(By.TAG_NAME, "img")
            for img_idx, img in enumerate(image_elements):
                img_url = img.get_attribute("src")
                if img_url:
                    img_data = requests.get(img_url).content
                    image_filename = f"content_{idx + 1}_image_{img_idx + 1}.jpg"
                    with open(image_filename, "wb") as img_file:
                        img_file.write(img_data)
                    print(f"Saved image to {image_filename}.")

            # 원래 페이지로 돌아가기
            driver.back()

            # 자립정보 조회 페이지로 재진입 (JavaScript 사용)
            driver.execute_script(script)
            gallery_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
            )

            # 사람처럼 보이게 대기
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing link {idx + 1}: {e}")
            break

except Exception as e:
    print(f"Error finding gallery_list or list items: {e}")

# 브라우저 종료
driver.quit()


No alert found or error occurred: Message: 

gallery_list found.
Saved text content to content_1.txt.
Saved text content to content_2.txt.
Saved image to content_2_image_1.jpg.
Saved text content to content_3.txt.
Saved image to content_3_image_1.jpg.
Saved text content to content_4.txt.
Saved image to content_4_image_1.jpg.
Saved text content to content_5.txt.
Saved image to content_5_image_1.jpg.
Saved text content to content_6.txt.
Saved image to content_6_image_1.jpg.
Saved image to content_6_image_2.jpg.
Saved image to content_6_image_3.jpg.
Saved text content to content_7.txt.
Saved image to content_7_image_1.jpg.
Saved text content to content_8.txt.
Saved image to content_8_image_1.jpg.
Saved image to content_8_image_2.jpg.
Saved image to content_8_image_3.jpg.
Saved image to content_8_image_4.jpg.
Saved image to content_8_image_5.jpg.
Saved image to content_8_image_6.jpg.
Saved image to content_8_image_7.jpg.
Saved image to content_8_image_8.jpg.
Saved image to content_8_image_


여기는 아직 진행 중이고 설치해야 할 사항이 많아서 수정 후에 적용시키려고 합니다!


**OCR**하는 과정에서 설치해야 할 것들!
1. 아래와 같이 pillow pytesseract 설치 필수
2. https://github.com/UB-Mannheim/tesseract/wiki#tesseract-installer-for-windows 링크 접속 후 exe 파일 다운 받고 해당 경로로 설정
3. 이미지 전처리를 이용해 OCR의 정확성 향상을 위해 cv2 설치하기

In [ ]:
pip install pillow pytesseract


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install opencv-python-headless


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from selenium import webdriver
import pytesseract
from PIL import Image
from io import BytesIO

# Tesseract OCR 경로 설정 (Windows에 설치된 경우 예시 경로)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# WebDriver 설정 (Chrome 예시)
driver = webdriver.Chrome()

# 메인 페이지로 이동
driver.get("https://jaripon.ncrc.or.kr/home/kor/main.do")

# 경고창 처리 (비정상적 접근 알림을 닫음)
try:
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert.accept()
    print("Alert accepted.")
except Exception as e:
    print(f"No alert found or error occurred: {e}")

# 자립정보 조회 페이지로 이동 (JavaScript 사용)
script = "fn_menu_move('/home/kor/support/projectMng/index.do', '3');"
driver.execute_script(script)

# 페이지 로딩 및 `div.gallery_list > ul.list` 요소 확인
try:
    gallery_list = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
    )
    print("gallery_list found.")

    # 각 `li > a` 요소 탐색 및 텍스트/이미지 저장 반복
    for idx in range(10):  # 예: 최대 10개의 항목 처리
        try:
            # 현재 `li > a` 요소들 로드
            li_elements = gallery_list.find_elements(By.CSS_SELECTOR, "li > a")

            # a 태그 클릭하여 새 페이지로 이동
            a_tag = li_elements[idx]
            driver.execute_script("arguments[0].click();", a_tag)

            # 새 페이지가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            # <div class="editor_view">에서 텍스트 및 이미지 URL 가져오기
            editor_view = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.editor_view"))
            )

            # 텍스트 내용 가져와서 저장
            text_content = editor_view.text
            text_filename = f"content_{idx + 1}.txt"
            with open(text_filename, "w", encoding="utf-8") as file:
                file.write(text_content)
            print(f"Saved text content to {text_filename}.")

            # 이미지 다운로드 및 저장
            image_elements = editor_view.find_elements(By.TAG_NAME, "img")
            for img_idx, img in enumerate(image_elements):
                img_url = img.get_attribute("src")
                if img_url:
                    # 이미지 다운로드
                    img_data = requests.get(img_url).content
                    image_filename = f"content_{idx + 1}_image_{img_idx + 1}.jpg"
                    with open(image_filename, "wb") as img_file:
                        img_file.write(img_data)
                    print(f"Saved image to {image_filename}.")

                    # 이미지에서 텍스트 추출 및 .txt 파일에 추가
                    image = Image.open(BytesIO(img_data))
                    ocr_text = pytesseract.image_to_string(image, lang="kor")  # 언어 설정 가능 (예: "kor" 또는 "eng+kor")
                    if ocr_text.strip():  # 추출된 텍스트가 있으면 추가
                        with open(text_filename, "a", encoding="utf-8") as file:
                            file.write("\n\n[Extracted text from image]\n")
                            file.write(ocr_text)
                        print(f"Extracted text from {image_filename} and appended to {text_filename}.")

            # 원래 페이지로 돌아가기
            driver.back()

            # 자립정보 조회 페이지로 재진입 (JavaScript 사용)
            driver.execute_script(script)
            gallery_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
            )

            # 사람처럼 보이게 대기
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing link {idx + 1}: {e}")
            break

except Exception as e:
    print(f"Error finding gallery_list or list items: {e}")

# 브라우저 종료
driver.quit()


No alert found or error occurred: Message: 

gallery_list found.
Saved text content to content_1.txt.
Saved text content to content_2.txt.
Saved image to content_2_image_1.jpg.
Extracted text from content_2_image_1.jpg and appended to content_2.txt.
Saved text content to content_3.txt.
Saved image to content_3_image_1.jpg.
Extracted text from content_3_image_1.jpg and appended to content_3.txt.
Saved text content to content_4.txt.
Saved image to content_4_image_1.jpg.
Saved text content to content_5.txt.
Saved image to content_5_image_1.jpg.
Extracted text from content_5_image_1.jpg and appended to content_5.txt.
Saved text content to content_6.txt.
Saved image to content_6_image_1.jpg.
Saved image to content_6_image_2.jpg.
Saved image to content_6_image_3.jpg.
Extracted text from content_6_image_3.jpg and appended to content_6.txt.
Saved text content to content_7.txt.
Saved image to content_7_image_1.jpg.
Extracted text from content_7_image_1.jpg and appended to content_7.txt.
Saved t

In [ ]:
import os
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from selenium import webdriver
import pytesseract
from PIL import Image
import cv2
import numpy as np
from io import BytesIO

# Tesseract OCR 경로 설정 (Windows에 설치된 경우 예시 경로)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# 이미지 전처리 함수
def preprocess_image(image):
    # 이미지를 NumPy 배열로 변환
    img_array = np.array(image)
    
    # 회색조로 변환
    gray_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    
    # Otsu's thresholding 적용 (이진화)
    _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 노이즈 제거 (옵션)
    denoised_img = cv2.fastNlMeansDenoising(binary_img, None, 30, 7, 21)

    # 처리된 이미지를 다시 PIL 이미지로 변환하여 반환
    return Image.fromarray(denoised_img)

# WebDriver 설정 (Chrome 예시)
driver = webdriver.Chrome()

# 메인 페이지로 이동
driver.get("https://jaripon.ncrc.or.kr/home/kor/main.do")

# 경고창 처리 (비정상적 접근 알림을 닫음)
try:
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert.accept()
    print("Alert accepted.")
except Exception as e:
    print(f"No alert found or error occurred: {e}")

# 자립정보 조회 페이지로 이동 (JavaScript 사용)
script = "fn_menu_move('/home/kor/support/projectMng/index.do', '3');"
driver.execute_script(script)

# 페이지 로딩 및 `div.gallery_list > ul.list` 요소 확인
try:
    gallery_list = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
    )
    print("gallery_list found.")

    # 각 `li > a` 요소 탐색 및 텍스트/이미지 저장 반복
    for idx in range(10):  # 예: 최대 10개의 항목 처리
        try:
            # 현재 `li > a` 요소들 로드
            li_elements = gallery_list.find_elements(By.CSS_SELECTOR, "li > a")

            # a 태그 클릭하여 새 페이지로 이동
            a_tag = li_elements[idx]
            driver.execute_script("arguments[0].click();", a_tag)

            # 새 페이지가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            # <div class="editor_view">에서 텍스트 및 이미지 URL 가져오기
            editor_view = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.editor_view"))
            )

            # 텍스트 내용 가져와서 저장
            text_content = editor_view.text
            text_filename = f"content_{idx + 1}.txt"
            with open(text_filename, "w", encoding="utf-8") as file:
                file.write(text_content)
            print(f"Saved text content to {text_filename}.")

            # 이미지 다운로드 및 저장
            image_elements = editor_view.find_elements(By.TAG_NAME, "img")
            for img_idx, img in enumerate(image_elements):
                img_url = img.get_attribute("src")
                if img_url:
                    # 이미지 다운로드
                    img_data = requests.get(img_url).content
                    image_filename = f"content_{idx + 1}_image_{img_idx + 1}.jpg"
                    with open(image_filename, "wb") as img_file:
                        img_file.write(img_data)
                    print(f"Saved image to {image_filename}.")

                    # 이미지에서 텍스트 추출 및 .txt 파일에 추가
                    image = Image.open(BytesIO(img_data))
                    
                    # 전처리 후 텍스트 추출
                    preprocessed_image = preprocess_image(image)
                    ocr_text = pytesseract.image_to_string(preprocessed_image, lang="kor")  # 한글 인식
                    if ocr_text.strip():  # 추출된 텍스트가 있으면 추가
                        with open(text_filename, "a", encoding="utf-8") as file:
                            file.write("\n\n[Extracted text from image]\n")
                            file.write(ocr_text)
                        print(f"Extracted text from {image_filename} and appended to {text_filename}.")

            # 원래 페이지로 돌아가기
            driver.back()

            # 자립정보 조회 페이지로 재진입 (JavaScript 사용)
            driver.execute_script(script)
            gallery_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gallery_list ul.list"))
            )

            # 사람처럼 보이게 대기
            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"Error processing link {idx + 1}: {e}")
            break

except Exception as e:
    print(f"Error finding gallery_list or list items: {e}")

# 브라우저 종료
driver.quit()


ModuleNotFoundError: No module named 'cv2'